# Generador StyleGAN ADA PyTorch

In [ ]:
#@title Montar Google Drive
#@markdown Monte Google Drive para cargar modelos previamente entrenados y guardar los resultados.

#@markdown Después de ejecutar esta celda, obtendrá el enlace. Haga click en el enlace, otorgue acceso a su Drive y copie el código de autenticación.

#@markdown Pegue el código en la entrada a continuación y presione Enter
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Instalación
#@markdown StyleGAN2-ADA será instalada en su Google Drive para acelerar el proceso de entrenamiento.

#@markdown Ejecute esta celda. Si ya ha instalado el repositorio, se omitirá el proceso de instalación y actualizará el directorio del repositorio. Si no lo ha instalado, instalará todos los archivos necesarios.
import os
import shlex
import numpy as np

if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

# Generación

In [ ]:
#@title Generación de Imágenes
#@markdown Directorio para guardar las imágenes generadas
outdir = "" #@param {type: "string"}
#@markdown Ruta al modelo preentrenado. La forma más sencilla es buscar el archivo en el directorio del Drive que se encuentra en el panel de la izquierda, hacer clic en los tres puntos y escoger la opción `copy path`, luego pegarlo aquí:
network = "" #@param {type: "string"}
#@markdown ---
#@markdown **Parámetros de Generación**

#@markdown El "truncamiento", digamos que *redondea o trunca* los valores del espacio latente. Esto puede tener un efecto sutil o dramático en sus imágenes dependiendo del valor que utilice. Cuanto menor sea el número, más realistas deberían aparecer sus imágenes, pero esto también afectará la diversidad. La mayoría de la gente elige entre 0.5 y 1.0, pero técnicamente es infinito.
truncation =  #@param {type: "number"}
#@markdown Esto le permite elegir semillas (seeds) aleatorias del modelo. Recuerde que nuestra entrada a StyleGAN es una matriz de 512 dimensiones. Estas semillas generarán esos 512 valores. Cada semilla generará una matriz aleatoria diferente. El mismo valor de inicialización también generará siempre la misma matriz aleatoria, por lo que luego podemos usarla para otros fines, como la interpolación.
#@markdown 
#@markdown Proporcione un número, una lista de enteros separados por comas o un rango entre valores utilizando un guión.
seeds = "" #@param {type: "string"}
#@markdown Generación de imágenes aleatorias (se ignorará el parámetro de semillas)
gen_random = False #@param {type: "boolean"}
#@markdown Cantidad de imágenes aleatorias a generar
n_imgs =  #@param {type: "integer"}
#@markdown ---
#@markdown **Generación de imágenes no-cuadradas**

#@markdown Es posible hacer que el modelo genere imágenes que no sean cuadradas. Esto no es tan bueno como entrenar un modelo rectangular, pero con el modelo correcto aún puede verse bien.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Ancho
width = 1920 #@param {type: "integer"}
#@markdown Alto
height = 1080 #@param {type: "integer"}
#@markdown Estilo de relleno para aplicar en el espacio adicional.
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = ','.join(str(s) for s in list(set(list(np.random.randint(4294967295, size=n_imgs)))))
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''


outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare}

In [ ]:
#@title Truncamiento Transversal

#@markdown A continuación, puede tomar una semilla y ver los cambios en ella en cualquier cantidad de truncamiento. -1 a 1 serán imágenes bastante realistas, pero cuanto más te alejas, más extrañas se vuelven.

#@markdown Directorio para guardar las imágenes generadas
outdir = "" #@param {type: "string"}
#@markdown Ruta al modelo preentrenado. La forma más sencilla es buscar el archivo en el directorio del Drive que se encuentra en el panel de la izquierda, hacer clic en los tres puntos y escoger la opción `copy path`, luego pegarlo aquí:
network = "" #@param {type: "string"}

#@markdown Escoja solo una semilla. Elija una favorita de sus imágenes generadas.
seed = 42  #@param {type: "integer"}
#@markdown Valor de truncamiento inicial.
start = -1  #@param {type: "number"}
#@markdown Valor donde se detiene el truncamiento. Debe ser mayor que el valor inicial. (Ocurrirá un error si no es así)
stop = 1  #@param {type: "number"}
#@markdown Cuánto debe incrementar cada fotograma el valor de truncamiento. Coloque un valor muy pequeño si desea una interpolación larga y lenta. (parada-inicio / incremento = fotogramas totales)
increment = 0.01  #@param {type: "number"}

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --process="truncation" --outdir={outdir} --start={start} --stop={stop} --increment={increment} --seeds={seed} --network={network}

In [ ]:
#@title Interpolación
#@markdown Directorio para guardar las imágenes generadas
outdir = "" #@param {type: "string"}
#@markdown Ruta al modelo preentrenado. La forma más sencilla es buscar el archivo en el directorio del Drive que se encuentra en el panel de la izquierda, hacer clic en los tres puntos y escoger la opción `copy path`, luego pegarlo aquí:
network = "" #@param {type: "string"}
#@markdown ---
#@markdown **Parámetros de Generación**

#@markdown El "truncamiento", digamos que redondea o trunca los valores del espacio latente. Esto puede tener un efecto sutil o dramático en sus imágenes dependiendo del valor que utilice. Cuanto menor sea el número, más realistas deberían aparecer sus imágenes, pero esto también afectará la diversidad. La mayoría de la gente elige entre 0.5 y 1.0, pero técnicamente es infinito.
truncation =  0.9#@param {type: "number"}
#@markdown Esto le permite elegir semillas (seeds) aleatorias del modelo. Recuerde que nuestra entrada a StyleGAN es una matriz de 512 dimensiones. Estas semillas generarán esos 512 valores. Cada semilla generará una matriz aleatoria diferente. El mismo valor de inicialización también generará siempre la misma matriz aleatoria, por lo que luego podemos usarla para otros fines, como la interpolación.
#@markdown 
#@markdown Proporcione un número, una lista de enteros separados por comas o un rango entre valores utilizando un guión.
seeds = "" #@param {type: "string"}
#@markdown Generación de imágenes aleatorias (se ignorará el parámetro de semillas)
gen_random = False #@param {type: "boolean"}
#@markdown Cantidad de imágenes aleatorias a generar
n_imgs =  #@param {type: "integer"}
#@markdown Interpolación en bucle (loop)
gen_loop = False #@param {type: "boolean"}
#@markdown ---
#@markdown **Parámetros de Interpolación**
#@markdown
#@markdown Número de cuadros (frames)
frames =  #@param {type: "integer"}

#@markdown Tipo de Interpolación
interpolation = 'linear' #@param ['linear', 'slerp', 'noiseloop', 'circularloop']
#@markdown Espacio latente
space = 'z' #@param ['z', 'w']
#@markdown ---
#@markdown **Generación de imágenes no-cuadradas**

#@markdown Es posible hacer que el modelo genere imágenes que no sean cuadradas. Esto no es tan bueno como entrenar un modelo rectangular, pero con el modelo correcto aún puede verse bien.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Ancho
width = 1920 #@param {type: "integer"}
#@markdown Alto
height = 1080 #@param {type: "integer"}
#@markdown Estilo de relleno para aplicar en el espacio adicional.
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = list(set(list(np.random.randint(4294967295, size=n_imgs))))
  if gen_loop:
    seeds.append(seeds[-1])
  seeds = ','.join(str(s) for s in seeds)
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare} --space={space} --process="interpolation" --interpolation={interpolation} --frames={frames}

In [ ]:
#@title Bucles de Interpolación
#@markdown Directorio para guardar las imágenes generadas
outdir = "" #@param {type: "string"}
#@markdown Ruta al modelo preentrenado. La forma más sencilla es buscar el archivo en el directorio del Drive que se encuentra en el panel de la izquierda, hacer clic en los tres puntos y escoger la opción `copy path`, luego pegarlo aquí:
network = "" #@param {type: "string"}
#@markdown ---
#@markdown **Parámetros de Generación**

#@markdown El "truncamiento", digamos que redondea o trunca los valores del espacio latente. Esto puede tener un efecto sutil o dramático en sus imágenes dependiendo del valor que utilice. Cuanto menor sea el número, más realistas deberían aparecer sus imágenes, pero esto también afectará la diversidad. La mayoría de la gente elige entre 0.5 y 1.0, pero técnicamente es infinito.
truncation =  1#@param {type: "number"}

#@markdown ---
#@markdown **Parámetros de Interpolación**

#@markdown Tipos de interpolación
interpolation = 'circularloop' #@param ['noiseloop', 'circularloop']
#@markdown Número de cuadros (frames)
frames =  360#@param {type: "integer"}
#@markdown Esto controla qué tan "amplio" es el bucle. Hágalo más pequeño para mostrar una gama menos diversa de muestras o más grande para cubrir muchas muestras. Estos `frames` adicionales pueden ayudar a determinar qué tan rápido se siente el video.
diameter =  50#@param {type: "number"}
#@markdown Lugar de inicio en el espacio z. Nota: este valor no tiene nada que ver con las semillas que usa para generar imágenes. Solo te permite aleatorizar tu punto de inicio.
random_seed =  #@param {type: "integer"}

#@markdown ---
#@markdown **Generación de imágenes no-cuadradas**

#@markdown Es posible hacer que el modelo genere imágenes que no sean cuadradas. Esto no es tan bueno como entrenar un modelo rectangular, pero con el modelo correcto aún puede verse bien.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Ancho
width = 1920 #@param {type: "integer"}
#@markdown Alto
height = 1080 #@param {type: "integer"}
#@markdown Estilo de relleno para aplicar en el espacio adicional.
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --network={network} {nonsquare} --process="interpolation" --interpolation={interpolation} --diameter={diameter} --random_seed={random_seed} --frames={frames}